In [1]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_key="b36879e47f2d42109a3d54edb22f78c3",
    azure_endpoint="https://ecotribe.openai.azure.com/openai/deployments/Chat/chat/completions?api-version=2024-02-15-preview",
    api_version="2024-02-15-preview",
    model="gpt-4o-2024-08-06",
    temperature=0,
    max_tokens=None,
)

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import base64
from io import BytesIO

def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

image_path="/home/akshay/Documents/conxai/Hacknow_SickAG/hacknow-ecotribe/apple_rotten.jpg"
image_base64 = image_to_base64(image_path)
prompt = ChatPromptTemplate.from_messages([
    ("system", '''
Imagine yourself as a gas detector which can identify if a fruit is releasing any gases which can indicate if it is fresh, ripen or rotten.
             Due to unavailability of gas detector sensors, you will be given an image as input, and your job is to guess which gases might have been released.
        Your task is to return a json object with following keys:
        {{
        level_of_ethylene: str # options: ["excessive", "high", "normal", "not_detected"]. Excessive means already ripened
        level_of_CO2: str # options: ["excessive", "high", "normal", "not_detected"]. Spike in CO2 indicates reduced shelf life
        level_of_methane: str # options: ["excessive", "high", "normal", "not_detected"]. If methane detected, fruit is spoiled
        level_of_ammonia: str # option: ["excessive", "high", "normal", "not_detected"]. The presence of ammonia indicates significant spoilage and bacterial activity.
        level_of_hydrogenSulfide: str # option: ["excessive", "high", "normal", "not_detected"]
             }}
     
     Be creative with the responses as these information would help the user to identify if the fruit is fresh, partially-ripen, fully-ripen, partially-rotten, or fully-roten.
    '''),
    ("human", "Please analyze this fruit image and return only JSON object as mentioned in the instructions. Reevaluate again if you are returning only JSON object or not."),
    ("human", f"![fruit_image](data:image/jpeg;base64,{image_base64})")

])

output_parser = JsonOutputParser()

response = llm.invoke(prompt.format(), response_format={"type": "json_object"})

In [5]:
response

AIMessage(content='{\n    "has_spoilage": "yes",\n    "description": "The fruit shows signs of external spoilage with visible bruising and discoloration. The extent of spoilage appears moderate, indicating that the fruit is partially rotten."\n}', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 14541, 'total_tokens': 14590, 'completion_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_80a1bad4c7', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': 

In [12]:
fix_json(response.content)

'{  "object_name": "apple",  "color": "red",  "surface_texture": "smooth",  "has_mould_spot": "no",  "shape": "normal"}'

In [8]:
output_parser = JsonOutputParser()
output_parser.parse(response.content)

{'level_of_ethylene': 'high',
 'level_of_CO2': 'normal',
 'level_of_methane': 'not_detected',
 'level_of_ammonia': 'not_detected',
 'level_of_hydrogenSulfide': 'not_detected'}